In [ ]:
import pandas as pd

In [ ]:
# prepare dati affluenza
#affluenza = pd.read_csv('../data/affluenzaScuole1617.csv') # actually comma separated

afflStatali = pd.read_csv('../data/stat_affluenzaScuole1617.csv')
afflParitarie = pd.read_csv('../data/parit_affluenzaScuole1617.csv')
affluenza = pd.concat([afflStatali, afflParitarie], axis=0)

In [ ]:
afflStatali

In [ ]:
# Join con affluenza:
inputAnagraficaFile = 'anagrScuoleMILANO_geoloc.csv'
anagraficaGeoloc = pd.read_csv(inputAnagraficaFile, sep = ';', decimal = ',').set_index('CODICESCUOLA')

anagraficaGeoloc = anagraficaGeoloc[~anagraficaGeoloc.Lat.isnull()] # drop missing geoloc
assert not any(anagraficaGeoloc.Lat.isnull()), 'Missing geoloc info'

# validate join 
anagraficaGeoloc.merge(affluenza, how='inner', left_index=True, right_on='CODICESCUOLA',
                sort=False, suffixes=('_anagr', '_affl'), copy=True, validate='1:m')
assert not any(anagraficaGeoloc.duplicated()), 'Duplicated students counts found'

In [ ]:
# check that all the student data corresponds to a known school
sourceFile = 'anagrScuoleAll.csv'
anagrAll = pd.read_csv(sourceFile, sep=';', decimal = ',', index_col='CODICESCUOLA')
assert all(affluenza.CODICESCUOLA.isin(anagrAll.index)), 'Unrecognized school codes'

# print the available fraction of student data
bFound  = anagraficaGeoloc.index.isin(affluenza.CODICESCUOLA.unique())
print(bFound.mean())

In [ ]:
# investigate missing values
missingSchools = anagraficaGeoloc[~bFound]
missingSchools.groupby('DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA').count()

In [ ]:
# drop missing schools in anagrafica by doing an inner join
alunniSum.to_frame()

In [ ]:
alunniSum = affluenza.groupby('CODICESCUOLA')['ALUNNI'].sum()
datiJoin = anagraficaGeoloc.join(alunniSum)

datiJoin = anagraficaGeoloc.merge(alunniSum.to_frame(), how='inner', left_index=True, right_index=True,
                sort=False, suffixes=('_anagr', '_affl'), copy=True, validate='1:1')

#datiJoin['ALUNNI'] = datiJoin['ALUNNI'].fillna(datiJoin['ALUNNI'].mean()) 



In [ ]:
datiJoin.to_csv('datiScuolaJoinedMilano.csv', sep = ';', decimal = ',')

In [ ]:
datiJoin.DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA.unique()
